In [ ]:
AWSTemplateFormatVersion: '2010-09-09'
Resources:
  LambdaExecutionRole:
    Type: 'AWS::IAM::Role'
    Properties:
      AssumeRolePolicyDocument:   #The trust policy that is associated with this role. Trust policies define which entities can assume the role.
        Version: '2012-10-17'
        Statement:
          - Effect: 'Allow'
            Principal:
              Service: 'lambda.amazonaws.com'
            Action: 'sts:AssumeRole'
      Policies:
        - PolicyName: 'LambdaExecutionPolicy_by_Juels'
          PolicyDocument:
            Version: '2012-10-17'
            Statement:
              - Effect: 'Allow'
                Action:
                  - 'logs:CreateLogGroup'
                  - 'logs:CreateLogStream'
                  - 'logs:PutLogEvents'
                Resource: 'arn:aws:logs:*:*:*'
              - Effect: 'Allow'
                Action:
                  - 'ec2:RunInstances'
                  - 'ec2:TerminateInstances'
                  - 'ec2:DescribeInstances'
                Resource: '*'
              - Effect: 'Allow'
                Action: 'lambda:InvokeFunction'
                Resource: '*'
  LambdaFunction:
    Type: 'AWS::Lambda::Function'
    Properties:
      Handler: 'index.my_lambda_handler'  #The name of the method within your code that Lambda calls to run your function.
      Role: !GetAtt LambdaExecutionRole.Arn
      Code:
        S3Bucket: s3://linuxjue
        S3Key: s3://linuxjue/lambda_code.zip
      Runtime: 'python3.8'
      Timeout: 900

  HttpApi:
    Type: 'AWS::ApiGatewayV2::Api'
    Properties:
      Name: 'LambdaHttpApi'
      ProtocolType: 'HTTP'

  HttpApiIntegration:
    Type: 'AWS::ApiGatewayV2::Integration'
    Properties:
      ApiId: !Ref HttpApi
      IntegrationType: 'AWS_PROXY'
      IntegrationUri: !Sub
        - arn:aws:apigateway:${Region}:lambda:path/2015-03-31/functions/${LambdaArn}/invocations
        - Region: !Ref 'AWS::Region'
          LambdaArn: !GetAtt LambdaFunction.Arn
      PayloadFormatVersion: '2.0'

  HttpApiRoute:
    Type: 'AWS::ApiGatewayV2::Route'
    Properties:
      ApiId: !Ref HttpApi
      RouteKey: 'POST /lambda'
      Target: !Sub 'integrations/${HttpApiIntegration}'

  HttpApistage:
    Type: 'AWS::ApiGatewayV2::Stage'
    Properties:
      ApiId: !Ref HttpApi
      StageName: '$default'
      AutoDeploy: true

  LambdaApiPermission:
    Type: 'AWS::Lambda::Permission'
    Properties:
      Action: 'lambda:InvokeFunction'
      FunctionName: !GetAtt LambdaFunction.Arn
      Principal: 'apigateway.amazonaws.com'
      SourceArn: !Sub
        - arn:aws:execute-api:${Region}:${AccountId}:${ApiId}/*
        - Region: !Ref 'AWS::Region'
          AccountId: !Ref 'AWS::AccountId'
          ApiId: !Ref HttpApi

Outputs:
  ApiUrl:
    Description: 'HTTP API URL'
    Value: !Sub 'https://${HttpApi}.execute-api.${AWS::Region}.amazonaws.com/lambda'


# AWS Lambda EC2 automated Management

This project contains an AWS Lambda function that creates a Linux EC2 instance and terminates it after 10 minutes. The Lambda function is triggered via an HTTP API created using API Gateway. The project includes a CloudFormation template to set up the necessary resources.

## Features

- **Automated EC2 Management**: Automatically create and terminate EC2 instances after 10 minutes.
- **HTTP API Integration**: Trigger the Lambda function using an HTTP API.
- **Time Savings**: Automate tedious tasks to save developer time.

## Architecture

1. **Lambda Function**: Creates an EC2 instance and schedules its termination.
2. **API Gateway**: Provides an HTTP endpoint to trigger the Lambda function.
3. **CloudFormation Template**: Sets up the Lambda function, API Gateway, and necessary IAM roles.

## Prerequisites

- AWS Account
- AWS CLI configured with appropriate permissions
- S3 bucket to store the Lambda function code

## Setup

### 1. Upload Lambda Function Code to S3

1. Create a zip file containing your Lambda function code.
2. Upload the zip file to an S3 bucket.

### 2. Deploy CloudFormation Stack

1. Save the CloudFormation template to a file, e.g., `template.yaml`.
2. Use the AWS CLI to deploy the CloudFormation stack:

   ```sh
   aws cloudformation create-stack --stack-name MyLambdaHttpApiStack --template-body file://template.yaml --capabilities CAPABILITY_NAMED_IAM